
## Document Retrieval 

### Objectives 
-   Concepts: Sparse vectorss  
-   Represent documents into vectors using Gensim. - TFIDF Models
-   Application Document Similarity - To use Gensim to compute similarities between documents.
-   Application TDocument Retrieval - To use Gensim to rank documents based on relevance to a query.

### Sparse Vectors


To convert a piece of text into a vector, we need to first determine the
dimension of the vector, or in other words the number of components of
the vector. Generally, the dimension of the vectors representing
documents (and thus the dimension of the vector space) is the same as
the vocabulary size, that is, each unique word corresponds to an entry
of the vectors. Here vocabulary refers to the set of all unique words in
a corpus.

While finding out the vocabulary size is not a problem — you may want to
think about how to do this — the real problem is that for a real world
corpus, the vocabulary is usually very large. It is not uncommon to have
millions of words in a vocabulary. If we represent each document as a
very high-dimensional vector, we will need a lot of space to store these
vectors either in memory or on disk. In reality, however, each document
contains only a relatively small set of words, so the vector used to
represent a document has most entries equal to zero and a small subset
of entries with non-zero values. When we store such kind of vectors in a
computer, we typically use a *sparse vector* representation. In this
representation, we only need to specify the values of those non-zero
entries of a vector.

For example, suppose our vocabulary has 10 words. Let us look at the
following vector: $$\begin{aligned}
v & = &
\begin{pmatrix}
1 \\
0 \\
0 \\
2 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0.5 \\
\end{pmatrix}.\end{aligned}$$ To store this vector in its original form,
we need to store 10 numbers, each corresponding to one entry of the
vector. A sparse vector representation stores only the non-zero entries
as follows: $$\begin{aligned}
v & = & \left( (0, 1), (3, 2), (9, 0.5) \right).\end{aligned}$$ Here the
sparse vector is a list of pairs. For each pair, the first number is an
ID or index indicating a particular entry of the original vector. For
example, $(0, 1)$ indicates that the first entry of the original $v$ has
a value of 1, and $(9, 0.5)$ indicates that the tenth entry of the
original $v$ has a value of 0.5. We can see that the amount of space
needed to store this sparse vector is now reduced to twice the number of
non-zero entries of the original vector.

### Create the sparse vectors for the SGNews dataset

-  Create the corpus, sg_corpus using gensim 

-  Create a dictionary for the SGNews_Apr2012 corpus

-  Create the processed sg_corpus and store in sg_stem. 




In [1]:
# Enter your code here
# Creating a dictionary from SGNews Apr2012.
#import nltk
from nltk.corpus import PlaintextCorpusReader

import gensim
from gensim import corpora

from gensim.parsing.porter import PorterStemmer
stemmer = PorterStemmer()

sg_corpus = PlaintextCorpusReader('data/SGNews_Apr2012', '.+\.txt')
sg_fids = sg_corpus.fileids()

sg_docs=[]
for file in sg_fids:
    with open('data/SGNews_Apr2012/'+file, 'r') as file_to_read:
        doc = file_to_read.read()
        sg_docs.append(doc)
sg_docs=[list(gensim.utils.tokenize(f)) for f in sg_docs]
sg_lower = [[w.lower() for w in doc] for doc in sg_docs]
sg_stop = [[w for w in doc if w not in gensim.parsing.preprocessing.STOPWORDS] for doc in sg_lower]
sg_stem = [[stemmer.stem(w) for w in doc] for doc in sg_stop]


## Document Representation as dense vectors

### Creating a Dictionary from a Corpus 

Now think about converting all documents in a corpus into vectors. We
need to map each unique word in the vocabulary of this corpus to an ID
or index first. These mappings from words to IDs are represented by a
class called `Dictionary` in Gensim.

In order to work on text documents, Gensim requires the words (aka tokens) be converted to unique ids. In order to achieve that, Gensim lets you create a Dictionary object that maps each word to a unique id.


Assume that `sg_stem` represents our corpus after several steps of
pre-processing. Note that `sg_stem` is a list of lists. The code below shows how a dictionary is created from
`sg_stem`.

In [ ]:
#Create dictionary
sg_dictionary = corpora.Dictionary(sg_stem)

print(sg_dictionary)



We need to import Gensim first and then import the `corpora` module from
Gensim. The `print` function is used to display the generated
dictionary. If you have correctedly performed the pre-processing steps such as stop word removal, you should see that there are <b> 6337 unique tokens (words) </b> in the
corpus, and only a few of them are shown.

What if we want to see all the mappings from tokens to IDs in this
dictionary? We can use `dictionary.token2id` to obtain a `dict` object which contains all the
mappings. See below. `dict` is built-in data type (and
class) in Python. It can be used to store mappings from some elements to
some other elements. For those of you who have programming background,
this is similar to a hash table (e.g., the HashMap class in Java). The
code below shows how we use a variable
`token_to_id` to store the mappings from tokens to IDs.

In [ ]:
token_to_id = sg_dictionary.token2id
print(type(token_to_id))
print(token_to_id)

For example, "letter" corresponds to ID 71, indicating that the $71^{\text{th}}$
entry of a vector in this vector space represents "letter". (Note that since we have performed stemming, not
all tokens are meaningful English words.)

In [ ]:
print(sg_dictionary.token2id['banner'])

Next, we use the function doc2bow to convert doc to another list which we call vec (to indicate that this is a vector). (Here bow stands for bag of words, meaning that the order of the words in the original document is ignored and we treat a document as a bag of words without any order.)
As we can see below, the generated vec for the second file is a list where each element is a pair of numbers. For example, the first element of vec is (3, 1), and the second element of vec is (8, 1).

In [ ]:
# Converting all documents in SGNews Apr2012 to a list of sparse vectors. Store in sg_vecs variable
sg_vecs = [sg_dictionary.doc2bow(doc) for doc in sg_stem]
print(sg_vecs[1][0:10]) #prints the 10 words in the second file


### Application - Document Similarity
#### Computing Similarities between Documents


Gensim has built-in functions to compute cosine similarities between
documents. Let us use a simple example to see how it works. We first
define a very small corpus with only two documents, represented as
sparse vectors below. We can see that the vector
space of this corpus has a dimension of 4 (because we see only the IDs
0, 1, 2 and 3 in the vectors).

Given this corpus, we first build an index for more efficient
computation later on, as shown below. Note that to
build an index from a list of sparse vectors, we need to also give the
dimension of the vector space, thus the argument `4` in
`similarities.SparseMatrixSimilarity(toy_corpus, 4)`.

You do not need to understand full details of how this index looks like and
exactly why we need such an index. If you are interested, you can read
up on *inverted index* (<http://en.wikipedia.org/wiki/Inverted_index>).

In [ ]:
from gensim import similarities

toy_corpus = [[(0,1),(1,1),(2,1)],[(1,2),(3,1)]]
index = similarities.SparseMatrixSimilarity(toy_corpus,4)

Assume that we have a new document represented as `[(0, 1)]`. That is,
this new document has only a single word corresponding to ID 0. To
compute the similarities between this document and all documents in
`toy_corpus`, refer to the code below. Here `index[test_doc]`
returns all the cosine similarity measures between the document
`test_doc` and documents in `toy_corpus`.

In [ ]:
test_doc = [(0,1)]
sims = index[test_doc]

To display these similarity measures, we can use the code shown below.


In [ ]:
print(list(enumerate(sims)))

What the output shows is that the cosine similarity
between `test_doc` and the first document in the index is around 0.577,
while the similarity between `test_doc` and the second document is 0.0.

We can try another test document, as shown below.


In [ ]:
test_doc = [(1,1), (3,1)]
sims = index[test_doc]
print(list(enumerate(sims)))

### Represent documents as TFIDF Vectors  - TF-IDF Weighting (Optional)

We can use Gensim to help compute inverse document frequencies (IDFs)
and use them to re-assign weights to document vectors. To do this, we
need to import `models` from Gensim first. See below.


In [ ]:
from gensim import models
tfidf = models.TfidfModel(toy_corpus)


The code above simply computes the inverse document
frequencies for all words in the input corpus. The formula it uses to
compute IDF is the following: $$\begin{aligned}
\textit{idf}_w  & = & \log_2 \frac{D}{\textit{df}_w},\end{aligned}$$
where $D$ is the total number of documents in the corpus and
$\textit{df}_w$ is the document frequency of word $w$, i.e., the number
of documents in which $w$ occurs.

Recall that our toy corpus has 4 words. Using
the formula above, we can manually calculate the IDF values of the four
words. For example, for the word with ID 0, its document frequency is 1,
and the corpus has 2 documents, so its inverse document frequency is
$\log_2 2 = 1$. For the word with ID 1, its document frequency is 2 and
the corpus also has 2 documents, so its inverse document frequency is
$\log_2 1 = 0$.

Now let us define a test document as follows:

In [ ]:
test_doc = [(0,1), (1,1), (2,1), (3,1)]


This document contains all the words in the vocabulary of the toy
corpus, and every word occurs only once. We can use the `tfidf` object
to transform `test_doc`, as shown in below.


In [ ]:
print(tfidf[test_doc])

First of all, we can see that in the transformed vector, we no longer
see an entry corresponding to the word with ID 1. This makes sense
because the IDF of this word is 0, and therefore after TF-IDF weighting,
this word has a value of 0 in the vector and does not need to appear in
the sparse vector representation. However, for all the other words,
their IDF values are 1. Shouldn’t the converted vector be
`[(0, 1), (2, 1), (3, 1)]`?

The reason we do not see `[(0, 1), (2, 1), (3, 1)]` is that the `tfidf`
object automatically *normalizes* the vectors when it transforms them.
When a vector is normalized, the value of each entry is divided by the
*norm* of this vector, such that the norm of the new vector is exactly
1.

So for the sparse vector `[(0, 1), (2, 1), (3, 1)]`, mathematically it
is represented as $$\begin{aligned}
v & = &
\begin{pmatrix}
1 \\
0 \\
1 \\
1 \\
\end{pmatrix}.\end{aligned}$$ We can see that when we normalize $v$, it
becomes $$\begin{aligned}
v' & = & \frac{v}{\|v\|} \\
& = & \frac{v}{\sqrt{3}} \\
& = &
\begin{pmatrix}
0.5774 \\
0 \\
0.5774 \\
0.5774 \\
\end{pmatrix}.\end{aligned}$$

Normalization does not affect cosine similarities between two vectors.
In fact, if all vectors have been normalized, then to compute their
cosine similarities, we only need to compute their *dot products*
because every vector’s norm is 1.

### SG_News Corpus to TFIDF Vectors 

1. Based on the example above, create a `TfidfModel` object from
the `SGNews_Apr2012` corpus
2. Transform all the document
vectors in this corpus with TF-IDF weighting


In [ ]:
#Convert the sg_vecs to tfidf vectors using TfidfModel function

sg_tfidf = models.TfidfModel(sg_vecs)
sg_vecs_with_tfidf = [sg_tfidf[vec] for vec in sg_vecs]

In [ ]:
print(sg_vecs_with_tfidf[1][0:10]) #prints the 10 words in the second file

### Application - Document Retrieval
#### Matching and Ranking the documents 

The last part (and maybe the most exciting part as well) of this lab is
to see how cosine similarity with TF-IDF weighting can be used for the
task of document retrieval. Given a query represented as a set of words,
the goal of document retrieval is to find a list of documents from a
corpus that are the most relevant to the query. This can be done by
ranking all the documents in the corpus based on their cosine
similarities to the query.

Recall the following variables have been defined:

-   `sg_dictionary`: The dictionary that contains the mappings from
    unique words in the `SGNews_Apr2012` corpus to integer IDs.

-   `sg_tfidf`: An `TfidfModel` object that stores the IDF values
    derived from this corpus and can be used to transform document
    vectors that have not been weighted with IDF.
-   `sg_vecs_with_tfidf`: A list of sparse vectors representing all the
    documents in this corpus. TF-IDF weighting has been performed.


Below are the steps:
1. Create index
2. Create the query vector after processing
3. Convert query tfidf vector
4. Apply the index on this query and get the sorted similarity score list of documents 


In [ ]:
print(sg_dictionary)

In [ ]:
#Why we need this? Why did we input 6337?
sg_index = similarities.SparseMatrixSimilarity(sg_vecs_with_tfidf, 6337)



Next, we define a query called `query1` that consists of two words:
“Malaysia” and “Singapore.” We need to use lowercase letters and perform
stemming such that the query is processed in the same way as the
documents in the corpus. We then convert the query into a sparse vector
using `doc2bow()`. After that, we perform TF-IDF weighting on the
vector. See below.

In [ ]:
query1 = [stemmer.stem('malaysia'), stemmer.stem('singapore')]
query1_vec = sg_dictionary.doc2bow(query1)
print(query1_vec)

In [ ]:
query1_vec_tfidf = sg_tfidf[query1_vec]
print(query1_vec_tfidf)


To verify that the transformation is correct,
the code below shows the IDs of the stemmed “malaysia” and
“singapore.” We can see that `query1_vec` contains both words, whereas
after TF-IDF weighting, `query1_vec_tfidf` contains only the stemmed “malaysia.” 
This is because
the word “Singapore” occurs in all the documents in this corpus, and
therefore its IDF value is 0.

In [ ]:
print(sg_dictionary.token2id[stemmer.stem('malaysia')])
print(sg_dictionary.token2id[stemmer.stem('singapore')])




Next, we compute the cosine similarities between the query and all
documents in the corpus. This is shown in below. You can
probably guess that the function `sorted` is used to sort a bunch of
elements. But what is `key=lambda item: -item[1]`? Do not worry too much
about this. You will not likely need to modify this part so it is OK if
you do not understand it. Essentially this code defines how the elements
should be sorted. You can see that they are sorted by the similarity
scores, which are always the second element in those pairs in round
brackets (thus `item[1]`). Also, sorting is in descending order (thus
the negative sign in front of `item[1]`).

In [ ]:
q1_sims = sg_index[query1_vec_tfidf]
q1_sorted_sims = sorted(enumerate(q1_sims), key = lambda item: -item[1])
print(q1_sorted_sims[0:10])

What the code shows is that the Rank 1 relevant document is
the document with an index of 206 in the corpus, the Rank 2 document is
document with index 138, and so on. To verify that these are indeed
documents related to the query, the code finds out the
original file IDs of those documents. Recall that the file IDs can be
retrieved using the function `fileids()`. By specifying the index into
the list of file IDs, we can get the original file names. Now open the
original text files with these IDs on your computer and check whether
they are indeed related to Malaysia. Also check whether the ones ranked
higher generally contain more occurrences of the word “Malaysia” and are
generally shorter. (Think about why.)